#Imports

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from unidecode import unidecode 
from sklearn.metrics import make_scorer, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import cross_val_score, KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import optuna
import joblib
import json
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature
from mlflow.data import from_pandas
from datetime import datetime
from tqdm import tqdm


#Funçoes

In [0]:
# Função que deixa colunas minúsculas
def normalize_columns(df):
    df.columns = [unidecode(col).lower() for col in df.columns]
    return df

# Função para normalizar os valores das colunas de texto
def normalize_text_values(df):
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].apply(lambda x: unidecode(x).lower() if isinstance(x, str) else x)
    return df

In [0]:
def quantile_loss_metric(y_true, y_pred, q=0.8):
    y_true = np.array(y_true).flatten()
    y_pred = np.array(y_pred).flatten()
    residual = y_true - y_pred
    return np.mean(np.maximum(q * residual, (q - 1) * residual))

def bias_metric(y_true, y_pred):
    return np.mean(y_pred - y_true)

def percentual_subestimativas(y_true, y_pred):
    return np.mean(y_pred < y_true)

In [0]:
def avaliar_com_mlflow(modelo, nome_modelo, dataset_nome, data_path, X_train, y_train, X_test, y_test):
    y_train = np.array(y_train).flatten()
    y_test = np.array(y_test).flatten()
    input_dataset = from_pandas(X_train, source=data_path, name="x_train")

    with mlflow.start_run(run_name=f"{nome_modelo} - {dataset_nome}"):
        modelo.fit(X_train, y_train)
        mlflow.log_input(input_dataset, context="training")

        y_pred_train = modelo.predict(X_train)
        y_pred_test = modelo.predict(X_test)

        # Métricas treino
        met_train = {
            "modelo": nome_modelo,
            "dataset": f"train-{dataset_nome}",
            "quantile_loss": quantile_loss_metric(y_train, y_pred_train),
            "mae": mean_absolute_error(y_train, y_pred_train),
            # "mape": mean_absolute_percentage_error(y_train[y_train>0], y_pred_train[y_pred_train>0]),
            "bias": bias_metric(y_train, y_pred_train),
            "pct_subestimado": percentual_subestimativas(y_train, y_pred_train)
        }

        # Métricas teste
        met_test = {
            "modelo": nome_modelo,
            "dataset": f"test-{dataset_nome}",
            "quantile_loss": quantile_loss_metric(y_test, y_pred_test),
            "mae": mean_absolute_error(y_test, y_pred_test),
            # "mape": mean_absolute_percentage_error(y_test[y_test>0], y_pred_test[y_pred_test>0]),
            "bias": bias_metric(y_test, y_pred_test),
            "pct_subestimado": percentual_subestimativas(y_test, y_pred_test)
        }

        # Parâmetros
        mlflow.log_params({
            'modelo': nome_modelo,
            "tipo_dataset": dataset_nome,
            'parametros': modelo.get_params()
        })

        mlflow.set_tag('Dataset', dataset_nome)

       # Métricas treino 
        for k, v in met_train.items():
            if k not in {"modelo", "dataset"}:
                mlflow.log_metric(f"{k}_train", v)

        # Métricas teste
        for k, v in met_test.items():
            if k not in {"modelo", "dataset"}:
                mlflow.log_metric(f"{k}_test", v)

        # Registro do modelo como artefato
        try:
            signature = infer_signature(X_train, modelo.predict(X_train))
            mlflow.sklearn.log_model(modelo, artifact_path="modelo", signature=signature)
        except Exception as e:
            print(f"[Aviso] Não foi possível salvar o modelo com mlflow.sklearn: {e}")

        return pd.DataFrame([met_train, met_test])

In [0]:
class EarlyStoppingCallback:
    def __init__(self, patience: int):
        self.patience = patience
        self.best_score = None
        self.num_no_improvement = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.FrozenTrial):
        current_score = study.best_value
        if self.best_score is None or current_score > self.best_score:
            self.best_score = current_score
            self.num_no_improvement = 0
        else:
            self.num_no_improvement += 1

        if self.num_no_improvement >= self.patience:
            study.stop()


#Lendo e preparando os dados

In [0]:
categ_cols = [
    'sexo',
    'signo',
    'fumante_x_regiao',
    'fumante_x_classe',
    'facebook_x_regiao',
    'facebook_x_classe',
    'fumante_x_facebook',
    'cat_fumante',
    'cat_facebook',
    'cat_idade',
    'cat_imc',
    'cat_filhos',
    'cat_regiao',
    'cat_classe'
]

In [0]:
# X_TRAIN
x_train = pd.read_parquet('data/x_prepared.parquet')

# Y_TRAIN
y_train = pd.read_excel('data/Seguro Saúde - Modelagem.xlsx', sheet_name='MODELAGEM')
y_train = normalize_columns(y_train) 
y_train = normalize_text_values(y_train)
y_train = y_train[['matricula', 'valor']]

# X_TEST
x_test = pd.read_csv('data/x_test_enginneered.csv')
for col in categ_cols:
    x_test[col] = x_test[col].astype('object')

# Y_TEST
y_test = pd.read_excel('data/Seguro Saúde - Teste Final.xlsx')[['MATRICULA', 'VALOR']]
y_test.columns = [x.lower() for x in y_test.columns]

# Recconhecendo os dados
print(f'X_TRAIN: [shape{x_train.shape}]')
display(x_train.head())
print(f'Y_TRAIN: [shape{y_train.shape}]')
display(y_train.head())
print(f'X_TEST: [shape{x_test.shape}]')
display(x_test.head())
print(f'Y_TEST: [shape{y_test.shape}]')
display(y_test.head())

In [0]:
sel_features = pd.read_csv('artifacts/selected_engineered_features.csv').features_selecionadas.tolist()
x_train_sel = x_train[sel_features]
x_train_sel.to_parquet('data/x_train_engineered_prepared_selected.parquet')

y_train = y_train[['valor']]

display(x_train_sel.head())
display(y_train.head())

In [0]:
full_pipeline = joblib.load("artifacts/pipeline_engineered_features.pkl")

x_test_prep = full_pipeline.transform(x_test)

feature_names = full_pipeline.get_feature_names_out()
x_test_prep = pd.DataFrame(x_test_prep, columns=feature_names, index=x_test.index)
x_test_prep = pd.concat([x_test[['matricula']], x_test_prep], axis=1)

print(f'X_TEST_PREPARED: [shape:{x_test_prep.shape}]')
display(x_test_prep.head())

In [0]:
x_test_prep_sel = x_test_prep[sel_features]
x_test_prep_sel.to_parquet('data/x_test_engineered_prepared_selected.parquet')

y_test = y_test[['valor']]

print(f'X_TEST_PREPARED_SEL: [shape:{x_test_prep_sel.shape}]')
display(x_test_prep_sel.head())
print(f'Y_TEST: [shape:{y_test.shape}]')
display(y_test.head())

In [0]:
high_corr = pd.read_csv('artifacts/high_corr_pairs.csv')
display(high_corr)

x_train_low_corr = x_train_sel.drop(columns=['num__idade', 'num__fumante_x_imc'])
x_test_low_corr = x_test_prep_sel.drop(columns=['num__idade', 'num__fumante_x_imc'])

x_train_low_corr.to_parquet('data/x_train_engineered_prepared_selected_low_corr.parquet')
x_test_low_corr.to_parquet('data/x_test_engineered_prepared_selected_low_corr.parquet')

print(x_train_low_corr.shape)
print(x_test_low_corr.shape)

#Escolha do modelo campeao

In [0]:
# Modelos sensíveis à multicolinearidade
modelos_nao_robustos = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso()
}

# Modelos robustos à multicolinearidade
modelos_robustos = {
    "DecisionTree": DecisionTreeRegressor(random_state=42),
    "RandomForest": RandomForestRegressor(n_jobs=-1, random_state=42),
    "XGBoost": XGBRegressor(n_jobs=-1, random_state=42),
    "LightGBM": LGBMRegressor(n_jobs=-1, random_state=42),
    "CatBoost": CatBoostRegressor(verbose=0, random_state=42),
    "HistGradientBoosting": HistGradientBoostingRegressor(random_state=42)
}

In [0]:
path = 'file:///dbfs/data/x_train_engineered_prepared_selected.parquet'
dataset_nome = 'ENGINEERED'
res_robustos = []
for nome_modelo, modelo in tqdm(modelos_robustos.items(), desc=f"MODELOS ROBUSTOS A MULTICOLINEARIDADE"):
    print(f'\n================================= {nome_modelo} =================================')
    df_metrics = avaliar_com_mlflow(modelo, nome_modelo, dataset_nome, path, x_train_sel, y_train, x_test_prep_sel, y_test)
    res_robustos.append(df_metrics)

df_robustos_resultados = pd.concat(res_robustos, ignore_index=True)

In [0]:
print(f'Média de gastos no conjunto de teste: R$ {round(y_test.valor.mean(), 2)}')

display(df_robustos_resultados.sort_values(by=["dataset", "quantile_loss", 'mae']))

In [0]:
path = 'data/x_train_engineered_prepared_selected_low_corr.parquet'
dataset_nome = 'ENGINEERED_LOW_CORR'
res_sensiveis = []
for nome, modelo in tqdm(modelos_nao_robustos.items(), desc=f"MODELOS SENSÍVEIS A MULTICOLINEARIDADE"):
    print(f'\n================================= {nome} =================================')
    resultado = avaliar_com_mlflow(modelo, nome, dataset_nome, path, x_train_low_corr, y_train, x_test_low_corr, y_test)
    res_sensiveis.append(resultado)

df_sensiveis_res = pd.concat(res_sensiveis, ignore_index=True)

In [0]:
print(f'Média de gastos no conjunto de teste: R$ {round(y_test.valor.mean(), 2)}')

display(df_sensiveis_res.sort_values(by=["dataset", "quantile_loss", 'mae']))

## Avaliação dos Modelos de Regressão

Nesta etapa, comparamos diferentes algoritmos de regressão para prever o valor gasto por clientes em um plano de saúde. A avaliação foi realizada com base em métricas que penalizam subestimações, além de considerar viés e erro absoluto médio.

### Métricas Utilizadas

- **Quantile Loss**: penaliza mais fortemente subestimações, conforme desejado para o negócio.
- **MAE (Mean Absolute Error)**: erro médio absoluto.
- **Bias**: média da diferença entre predição e valor real. Valores positivos indicam tendência a superestimar.
- **Percentual de Subestimações**: fração de predições abaixo do valor real.

---

### Top 3 Modelos com Melhor Desempenho

#### 1. Random Forest Regressor

- **Quantile Loss**: 429.88  
- **MAE**: 912.28  
- **Bias**: +87.54  
- **% Subestimado**: 31.5%

**Análise**: Este modelo apresenta o menor quantile loss, o que indica melhor performance em minimizar penalizações por subestimação. Além disso, possui viés positivo, o que é desejável no contexto, e um percentual de subestimações controlado.

#### 2. CatBoost Regressor

- **Quantile Loss**: 447.39  
- **MAE**: 855.02  
- **Bias**: –66.28  
- **% Subestimado**: 30.7%

**Análise**: Apresentou o menor MAE entre todos os modelos, com bom controle de subestimações. No entanto, o viés negativo pode representar um risco para decisões de negócios que não toleram subestimar valores.

#### 3. XGBoost Regressor

- **Quantile Loss**: 472.26  
- **MAE**: 932.89  
- **Bias**: –19.39  
- **% Subestimado**: 40.2%

**Análise**: Mostrou desempenho intermediário, com viés mais próximo de zero que o CatBoost. Porém, apresentou o maior percentual de subestimações entre os modelos do top 3.

---

### Modelo Campeão

**Random Forest Regressor** é o modelo escolhido como campeão desta etapa. Ele combina o menor valor de quantile loss com viés positivo e um percentual de subestimações relativamente baixo, equilibrando precisão e segurança para a tomada de decisão.


#Hypertunning do RandomForest

In [0]:
q_loss_scorer = make_scorer(quantile_loss_metric, greater_is_better=False)

def objective_random_forest(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 2000),
        "max_depth": trial.suggest_int("max_depth", 5, 40),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 20),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None]),
        "max_leaf_nodes": trial.suggest_int("max_leaf_nodes", 10, 100, step=10),
        "min_impurity_decrease": trial.suggest_float("min_impurity_decrease", 0.0, 1.0),
        "bootstrap": trial.suggest_categorical("bootstrap", [True, False]),
        "ccp_alpha": trial.suggest_float("ccp_alpha", 0.0, 0.1),
        "random_state": 42,
        "n_jobs": -1
    }

    model = RandomForestRegressor(**params)

    # Validação cruzada 10-fold
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(model, x_train_sel, y_train.squeeze(),
                             scoring=q_loss_scorer, cv=kf, n_jobs=-1)
    
    return scores.mean()

In [0]:
early_stopping = EarlyStoppingCallback(patience=40)

study = optuna.create_study(direction="maximize", study_name="random_forest_q_loss_opt")
study.optimize(objective_random_forest, n_trials=300, callbacks=[early_stopping], show_progress_bar=True)


with open("artifacts/best_random_forest_params_engineered_dataset.json", "w") as f:
    json.dump(study.best_trial.params, f)


In [0]:
with open("artifacts/best_random_forest_params_engineered_dataset.json", "r") as f:
    best_params = json.load(f)

best_params.update({
    "n_jobs": -1,
    "random_state": 42
})

modelo_final = RandomForestRegressor(**best_params)
modelo_final.fit(x_train_sel, y_train.squeeze())

In [0]:
joblib.dump(modelo_final, "artifacts/randmforest_engineered_features_q_loss.pkl")

In [0]:
feature_names = x_train_sel.columns if isinstance(x_train_sel, pd.DataFrame) else [f"feat_{i}" for i in range(x_train_sel.shape[1])]

importances = modelo_final.feature_importances_
plt.figure(figsize=(10, 6))
plt.barh(feature_names, importances)
plt.xlabel("Importance")
plt.title("RandomForest (Q-Loss) - Feature Importance")
plt.gca().invert_yaxis()
plt.tight_layout()

plt.savefig("plots/randomforest_engineered_features_q_loss_feature_importance.png")

plt.show()

# Performance Assessment

In [0]:
y_pred_test = modelo_final.predict(x_test_prep_sel)

relatorio_final = {
    "quantile_loss": quantile_loss_metric(y_test, y_pred_test),
    "mae": mean_absolute_error(y_test, y_pred_test),
    "mape": mean_absolute_percentage_error(y_test, y_pred_test),
    "bias": bias_metric(y_test.squeeze(), y_pred_test),
    "pct_subestimado": percentual_subestimativas(y_test.squeeze(), y_pred_test)
}

df_relatorio_final = pd.DataFrame([relatorio_final])
display(df_relatorio_final)

df_relatorio_final.to_csv("artifacts/randomforest_relatorio_q_loss_test.csv", index=False)

In [0]:
erro_abs = np.abs(y_test.squeeze() - y_pred_test)

plt.figure(figsize=(8, 4))
plt.hist(erro_abs, bins=30, edgecolor="k")
plt.title("Distribuição do Erro Absoluto (|y - y_pred|)")
plt.xlabel("Erro Absoluto")
plt.ylabel("Frequência")
plt.grid(True)
plt.tight_layout()

plt.savefig("plots/randomforest_q_loss_erro_abs.png")

plt.show()

In [0]:
q_loss_individual = np.maximum(0.8 * (y_test.squeeze() - y_pred_test), (0.8 - 1) * (y_test.squeeze() - y_pred_test))
mape_individual = np.abs((y_test.squeeze() - y_pred_test) / y_test.squeeze())
mae_individual = np.abs(y_test.squeeze() - y_pred_test)

df_erros = pd.DataFrame({
    "Q-Loss": q_loss_individual,
    "MAE": mae_individual,
    "MAPE": mape_individual
})

plt.figure(figsize=(10, 5))
df_erros.boxplot()
plt.title("Distribuição das Métricas Individuais no Conjunto de Teste")
plt.ylabel("Erro")
plt.grid(True)
plt.tight_layout()

plt.savefig("plots/randomforest_q_loss_erros_individuais.png")

plt.show()

In [0]:
plt.figure(figsize=(6, 6))
plt.scatter(y_test.squeeze(), y_pred_test, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', label="Ideal")
plt.xlabel("Valor Real")
plt.ylabel("Valor Predito")
plt.title("Real vs Predito - Teste")
plt.legend()
plt.grid(True)
plt.tight_layout()

plt.savefig("plots/randomforest_q_loss_real_vs_predito.png")

plt.show()

In [0]:
y_true = y_test.squeeze()
y_pred = y_pred_test.squeeze()

df_avaliacao = pd.DataFrame({
    "y_true": y_true,
    "y_pred": y_pred
})
df_avaliacao["erro"] = df_avaliacao["y_pred"] - df_avaliacao["y_true"]
df_avaliacao["mae"] = df_avaliacao["erro"].abs()

# Fateando target por quantis
df_avaliacao["faixa_valor"] = pd.qcut(df_avaliacao["y_true"], q=6, duplicates="drop")

# Calculando métricas por faixa
resumo_faixas = df_avaliacao.groupby("faixa_valor").agg(
    qtd=("y_true", "count"),
    media_real=("y_true", "mean"),
    mae_medio=("mae", "mean"),
    erro_medio=("erro", "mean")
).reset_index()

resumo_faixas["mae_pct"] = resumo_faixas["mae_medio"] / resumo_faixas["media_real"]

In [0]:
plt.figure(figsize=(10, 6))
plt.bar(resumo_faixas.index - 0.2, resumo_faixas["mae_medio"], width=0.4, label="MAE médio")
plt.bar(resumo_faixas.index + 0.2, resumo_faixas["erro_medio"], width=0.4, label="Erro médio (Viés)")
plt.xticks(resumo_faixas.index, resumo_faixas["faixa_valor"].astype(str), rotation=45, ha='right')
plt.xlabel("Faixa de valor real (quantis)")
plt.ylabel("Erro (R$)")
plt.title("MAE e Viés por Faixa de Valor Real")
plt.legend()
plt.tight_layout()
plt.grid(True)

plt.savefig("plots/randomforest_q_loss_mae_vies.png")

plt.show()

In [0]:
plt.figure(figsize=(10, 5))
plt.plot(resumo_faixas["mae_pct"], marker="o")
plt.xticks(resumo_faixas.index, resumo_faixas["faixa_valor"].astype(str), rotation=45, ha='right')
plt.ylabel("MAE percentual (MAE / Média da faixa)")
plt.xlabel("Faixa de valor real (quantis)")
plt.title("Erro Relativo por Faixa de Valor")
plt.grid(True)
plt.tight_layout()

plt.savefig("plots/randomforest_q_loss_mae_pct.png")

plt.show()

In [0]:
with mlflow.start_run(run_name="RandomForest - q-loss Optimized"):
    # Logar dataset de entrada
    mlflow.log_input(from_pandas(x_train_sel, source="data/x_train_engineered_prepared_selected.parquet"), context="training")

    # Logar o modelo
    mlflow.sklearn.log_model(modelo_final, artifact_path="modelo")

    # Logar parâmetros
    mlflow.log_params(best_params)

    # Logar métricas
    for k, v in relatorio_final.items():
        mlflow.log_metric(k, v)

    # Logar gráficos
    mlflow.log_artifact("plots/randomforest_engineered_features_q_loss_feature_importance.png")
    mlflow.log_artifact("plots/randomforest_q_loss_erro_abs.png")
    mlflow.log_artifact("plots/randomforest_q_loss_erros_individuais.png")
    mlflow.log_artifact("plots/randomforest_q_loss_real_vs_predito.png")
    mlflow.log_artifact("plots/randomforest_q_loss_mae_vies.png")
    mlflow.log_artifact("plots/randomforest_q_loss_mae_pct.png")


## Comparativo de Modelos RandomForest (MAE vs Quantile Loss)

### Resumo das Métricas no Conjunto de Teste

| Métrica           | RF - MAE Opt. | RF - QLoss Opt. |
|-------------------|---------------|------------------|
| MAE               | **808.80**    | 828.68           |
| Quantile Loss     | 433.70        | **430.42**       |
| MAPE              | 32.68%        | **31.40%**       |
| Bias              | -97.67        | **-53.58**       |
| % Subestimativas  | 24.41%        | **22.83%**       |

> O modelo otimizado para **Quantile Loss** apresentou melhor controle de viés e subestimativas, com desempenho semelhante ao modelo otimizado por MAE.

---

### Análise por Faixa de Valor

- Ambos os modelos possuem **erros absolutos maiores nas faixas altas**, como esperado.
- O **modelo QLoss** apresentou menor viés nas faixas superiores, o que é crítico para evitar subestimativas de valores relevantes.

---

### Interpretação de Negócio

- Para evitar riscos de **subestimativas em clientes de alto valor**, o modelo otimizado com **quantile loss é mais confiável**.
- Se o objetivo for apenas minimizar erro médio, o modelo MAE também é competitivo, mas com maior viés nas faixas superiores.

---

### Conclusão: Modelo Campeão

> **Modelo vencedor: `RandomForestRegressor` otimizado para Quantile Loss**

- Equilíbrio entre desempenho e segurança.
- Melhor controle de subestimativas.
- Viés reduzido e erros mais bem distribuídos entre faixas.

---

### Avaliação da Qualidade Absoluta

| Indicador                | Avaliação          |
|--------------------------|--------------------|
| MAE ≈ R$ 828             | **Aceitável a bom** |
| MAPE ≈ 31%               | **Moderado**       |
| Viés ≈ -53               | **Baixo e seguro** |
| % Subestimativas ≈ 23%   | **Seguro**         |

> Em resumo, o modelo é **bom**, seguro e confiável para aplicações práticas. A recomendação é sua adoção com monitoramento contínuo.
